In [1]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess

model_dir = "iic/SenseVoiceSmall"

ASR_model = AutoModel(
    model=model_dir,
    vad_model="fsmn-vad",
    vad_kwargs={"max_single_segment_time": 30000},
    device="cuda:1",
)

# en
res = ASR_model.generate(
    input=f"{ASR_model.model_path}/example/en.mp3",
    cache={},
    language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)

Notice: ffmpeg is not installed. torchaudio is used to load audio
If you want to use ffmpeg backend to load audio, please install it by:
	sudo apt install ffmpeg # ubuntu
	# brew install ffmpeg # mac
funasr version: 1.2.6.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.2.6


2025-07-01 19:55:02,624 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


2025-07-01 19:55:06,636 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
rtf_avg: 0.014: 100%|██████████| 1/1 [00:00<00:00, 11.03it/s]
rtf_avg: 0.013, time_speech:  7.176, time_escape: 0.093: 100%|██████████| 1/1 [00:00<00:00, 10.28it/s]

The tribal chieftain called for the boy and presented him with 50 pieces of gold.


In [2]:
import json
json_path = 'config/tfgridnet_embed_train.json'
config = json.load(open(json_path, 'r'))
import torch
from models import TFGridNetRealtimeEmbed

model = TFGridNetRealtimeEmbed(**config['model']['params'])

ckpt_path = 'runs/tfgridnet_embed_train/ABCS/checkpoints/epoch=4-step=23970.ckpt'
ckpt = torch.load(ckpt_path, map_location='cpu')
# remove the 'model.' prefix from the state_dict keys
ckpt['state_dict'] = {k.replace('model.', ''): v for k, v in ckpt['state_dict'].items()}
model.load_state_dict(ckpt['state_dict'], strict=True)
model.eval()

Net(
  (tfgridnet): TFGridNet(
    (enc): Encoder(
      (filterbank): STFTFB()
    )
    (dec): Decoder(
      (filterbank): STFTFB()
    )
    (conv): Sequential(
      (0): Conv2d(2, 64, kernel_size=(3, 3), stride=(1, 1), padding=(0, 1))
    )
    (embeds): ModuleList()
    (blocks): ModuleList(
      (0-2): 3 x GridNetBlock(
        (intra_norm): LayerNormalization4D(
          (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (intra_rnn): LSTM(64, 64, batch_first=True, bidirectional=True)
        (intra_linear): Linear(in_features=128, out_features=64, bias=True)
        (inter_norm): LayerNormalization4D(
          (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (inter_rnn): LSTM(64, 64, batch_first=True)
        (inter_linear): Linear(in_features=64, out_features=64, bias=True)
        (attn_conv_Q): Sequential(
          (0): Linear(in_features=64, out_features=24, bias=True)
          (1): PReLU(num_parameters=1)
    

In [3]:
from utils.vib_dataset import V2S_dataset
import matplotlib.pyplot as plt
import soundfile as sf
from tqdm import tqdm

def visualization_spec(audio_i, vibration_i, output):
    fig, axs = plt.subplots(3, 2, figsize=(8, 6))
    axs[0, 0].plot(audio_i, label='Audio Signal')
    axs[0, 1].specgram(audio_i, Fs=16000, NFFT=256, noverlap=128)

    axs[1, 0].plot(vibration_i, label='Vibration Signal', color='orange')
    axs[1, 1].specgram(vibration_i, Fs=16000, NFFT=256, noverlap=128, cmap='plasma')

    axs[2, 0].plot(output, label='Model Output', color='green')
    axs[2, 1].specgram(output, Fs=16000, NFFT=256, noverlap=128, cmap='viridis')
    plt.tight_layout()
    plt.show()

dataset = V2S_dataset()

device = 'cuda:1'
model.to(device)
gts = []; outputs = []; inputs = []
with torch.no_grad():
    for data in tqdm(dataset):
        audio = data['audio'][None, :]; vibration = data['vibration'][None, :]
        audio = torch.from_numpy(audio).to(device); vibration = torch.from_numpy(vibration).to(device)
        output = model(audio, vibration)
        audio_i = audio.cpu().numpy()[0]; vibration_i = vibration.cpu().numpy()[0]
        output = output.cpu().numpy()[0, 0]
        # visualization_spec(audio_i, vibration_i, output)
        sf.write('resources/audio.wav', audio_i, 16000)
        sf.write('resources/output.wav', output, 16000)
        res = ASR_model.generate(
        input='resources/output.wav',
        cache={},
        language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
        use_itn=True,
        batch_size_s=60,
        merge_vad=True,  #
        merge_length_s=15,
        )
        text = rich_transcription_postprocess(res[0]["text"])
        outputs.append(text)

        res = ASR_model.generate(
        input='resources/output.wav',
        cache={},
        language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
        use_itn=True,
        batch_size_s=60,
        merge_vad=True,  #
        merge_length_s=15,
        )
        text = rich_transcription_postprocess(res[0]["text"])
        inputs.append(text)
        gts.append(data['text'])

import jiwer 
gts = [x.replace(' ', '').strip() for x in gts]
# remove punctuation

print(gts, outputs, inputs)
cer = jiwer.cer(gts, outputs)
print(f"CER: {cer:.4f}")
cer = jiwer.cer(gts, inputs)
print(f"CER: {cer:.4f}")

rtf_avg: 0.002: 100%|██████████| 1/1 [00:00<00:00, 59.24it/s] 



rtf_avg: 0.009: 100%|██████████| 1/1 [00:00<00:00, 42.26it/s]
rtf_avg: 0.005, time_speech:  5.000, time_escape: 0.027: 100%|██████████| 1/1 [00:00<00:00, 36.47it/s]
rtf_avg: 0.002: 100%|██████████| 1/1 [00:00<00:00, 110.19it/s]



rtf_avg: 0.008: 100%|██████████| 1/1 [00:00<00:00, 48.05it/s]
rtf_avg: 0.005, time_speech:  5.000, time_escape: 0.023: 100%|██████████| 1/1 [00:00<00:00, 41.29it/s]
rtf_avg: 0.002: 100%|██████████| 1/1 [00:00<00:00, 57.35it/s] 



rtf_avg: 0.007: 100%|██████████| 1/1 [00:00<00:00, 43.82it/s]
rtf_avg: 0.005, time_speech:  5.000, time_escape: 0.025: 100%|██████████| 1/1 [00:00<00:00, 38.19it/s]
rtf_avg: 0.002: 100%|██████████| 1/1 [00:00<00:00, 109.75it/s]



rtf_avg: 0.007: 100%|██████████| 1/1 [00:00<00:00, 39.36it/s]
rtf_avg: 0.006, time_speech:  5.000, time_escape: 0.029: 100%|██████████| 1/1 [00:00<00:00, 33.89it/s]
rtf_avg: 0.002: 100%|██████████| 1/1 [00:00<00:00, 58.78it/s] 



rtf_avg: 0

['他的工作强度也加大许多', '他的工作强度也加大许多', '其他演讲者名单和小组会议将在未来几周公布', '其他演讲者名单和小组会议将在未来几周公布', '国内越来越多公司想挤进石墨烯行业', '国内越来越多公司想挤进石墨烯行业', '凭借劳动力的低成本优势', '凭借劳动力的低成本优势', '杨健兴奋道今天晚上中国军团注定要创造历史啊', '杨健兴奋道今天晚上中国军团注定要创造历史啊', '我猜很多人会被这个数字打击到', '我猜很多人会被这个数字打击到', '去年我国高耗能行业能耗占工业能耗的百分之一', '去年我国高耗能行业能耗占工业能耗的百分之一', '两人足迹可说是踏遍全球', '两人足迹可说是踏遍全球', '对于醉驾毒驾伤人的情况', '对于醉驾毒驾伤人的情况', '中国企业国际化分两个阵营', '中国企业国际化分两个阵营', '这一次挺进半决赛也全靠兰迪', '这一次挺进半决赛也全靠兰迪', '也会面临较高的资金成本压力', '也会面临较高的资金成本压力', '较全国性房地产销售品牌企业增幅低四点四四个百分点', '较全国性房地产销售品牌企业增幅低四点四四个百分点', '日前确定了主创班底', '记者采访本案制假嫌疑人之一杜某', '记者采访本案制假嫌疑人之一杜某', '合肥长丰男子左某某多次在深夜尾随独行女子', '合肥长丰男子左某某多次在深夜尾随独行女子', '其中国家补助资金三十亿元', '其中国家补助资金三十亿元', '一疑患艾滋嫌犯湖南常德闹市被擒拒捕咬伤民警', '一疑患艾滋嫌犯湖南常德闹市被擒拒捕咬伤民警', '此举也在圈内引发了不小的争议', '此举也在圈内引发了不小的争议', '爱上网且关心教育的朋友注意了', '爱上网且关心教育的朋友注意了', '在自己第四个本命年时', '在自己第四个本命年时', '或者冰箱内部空间按我的要求设计', '或者冰箱内部空间按我的要求设计', '董事会监事会独立董事都发挥着应有的作用', '董事会监事会独立董事都发挥着应有的作用', '并在舞台上首度谈起了去年纷扰了许多的禁赛风波', '并在舞台上首度谈起了去年纷扰了许多的禁赛风波', '互联网普及率为百分之四十八点八', '互联网普及率为百分之四十八点八', '以反映问题为由多次闹访男子被判三年', '以反映问题为由多次闹访男子被判三年', '耗

ValueError: one or more references are empty strings